In [ ]:
# This notebook can calculate an upper bound for the number of normal quaternionic matrices of order n.
# To do so, a class 'quaternionic structures' is established.
# It is also possible to discern the level s and parameters p = (d, f, r, v, t, u) or use partial quaternionic structures as input.
# All necessary notation and theoretical context can be found in the paper 
# "Normal Quaternionnic Matrices and Finitely Generated Witt Rings" of the same authors.
# Authors: Nico Lorenz (Ruhr-Universität Bochum, Germany) and Alexander Schönert (Technische Universität Dortmund, Germany)

In [8]:
# This cell contains all necessary functions inside and outside the class of quaternionic structures.
# The subsequent cell contains the main routine along with input options.

################################

#### IMPORTS

import numpy as np
import math
from itertools import chain, combinations
from functools import reduce
import time
from datetime import datetime
import os
import copy
from multiprocess import Pool
#the following alternative to multiprocess might be helpful for running in parallel on a windows system
#from multiprocessing import Pool, freeze_support
import sys
import shutil
import glob
import pickle


################################

#### SELECTION OF BASIS TRANSFORMATIONS 

## basis transformations are applied when the partial quaternionic matrix is defined up to entry (i,j)
## they are viewed as transformations of the basis (1, 2, 4, ... 2^(n-1)) which is written as a list [0, 1, ... n - 1]
## a sum of 2-powers 2^a + 2^b + ... is written as the list [a, b, ...]

## permutations, denoted perm_#1_#2_... of disjoint cycles of length #1, #2,..., counted alphabetically
def perm_3_2_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if (i > 2 or QS.s == 1) and j - 1 > i:
        permutation = [[i - 2, i, i - 1], [j - 1, j]]
        trafo = create_permutation(QS.n, permutation)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_2_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if (i > 2 or QS.s == 1) and j - 1 > i:
        permutation = [[i - 2, i - 1], [j - 1, j]]
        trafo = create_permutation(QS.n, permutation)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_2_b(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if j - 2 > i:
        permutation = [[i, i - 1], [j - 2, j - 1]]
        trafo = create_permutation(QS.n, permutation)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_2_c(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if (i > 2 or QS.s == 1) and (j - 2 > i):
        permutation = [[i - 2, i - 1], [j - 1, j]]
        trafo = create_permutation(QS.n, permutation)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_2_d(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if j - 1 > i:
        permutation = [[i - 1, i], [j - 1, j]]
        trafo = create_permutation(QS.n, permutation)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_5_2_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if j - 1 > i and (i > 4 or (QS.s == 1 and i == 4)):
        permutation = [[j, j - 1], [i, i - 2, i - 4, i - 1, i - 3]]
        trafo = create_permutation(QS.n, permutation)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_4_3_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if (j - 3 > i) and (i > 2 or QS.s == 1):
        permutation = [[j, j - 2, j - 1, j - 3], [i, i - 1, i - 2]]
        trafo = create_permutation(QS.n, permutation)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    permutation = [[i - 1, i]]
    trafo = create_permutation(QS.n, permutation)
    return apply_trafo_to_QM(QS.mat, trafo)

def perm_2_b(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if j - 1 > i:
        permutation = [[j - 1, j]]
        trafo = create_permutation(QS.n, permutation)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_c(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if j < QS.n - 2:
        permutation = [[j + 1, j + 2]]
        trafo = create_permutation(QS.n, permutation)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0
        
def perm_4_2_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if (j - 2 > i) and (i > 3 or (QS.s == 1 and i == 3)):
        permutation = [[j - 2, j], [i, i - 2, i - 1, i - 3]]
        trafo = create_permutation(QS.n, permutation)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

## combinations of permutations and row actions, the row action is always applied first
def perm_2_action_2_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if j - 2 > i:
        permutation = [[j - 2, j]]
        row_action = {x: [x] if x != i else [0, x] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_action_2_b(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if j - 2 > i:
        permutation = [[j - 2, j - 1]]
        row_action = {x: [x] if x != i else [0, x] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_action_2_c(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    permutation = [[i - 1, i]]
    row_action = {x: [x] if x != j - 1 else [x, x + 1] for x in range(QS.n)}
    trafo = create_trafo(QS.n, permutation, row_action)
    return apply_trafo_to_QM(QS.mat, trafo)

def perm_2_action_2_d(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if (j < QS.n - 1) and (QS.s == 1 or i > 2):
        permutation = [[j - 1, j + 1]]
        row_action = {x: [x] if x != i - 2 else [x, x + 1] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_action_2_e(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if j - 1 > i:
        permutation = [[j - 1, j]]
        row_action = {x: [x] if x != i - 1 else [x - 1, x] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_action_2_f(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if j > i + 2:
        # j-2, j-1, j -> j+(j-1), j-1, j-2
        permutation = [[j - 2, j]]
        row_action = {x: [x] if x != j - 2 else [x, x + 1] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_action_3_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if j > i + 2:
        permutation = [[j - 2, j - 3]]
        row_action = {x: [x] if x != i else [i, i - 1, i - 2] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_2_action_2_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if j > i + 1 and i > 3:
        permutation = [[j - 1, j], [i - 2, i]]
        row_action = {x: [x] if x != i - 3 else [x, x - 1] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_2_action_2_b(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if j > i + 3:
        permutation = [[j - 3, j - 1], [j - 2, j]]
        row_action = {x: [x] if x != i - 1 else [x, x - 1] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_2_action_3_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if j > i + 1 and (i > 3 or (QS.s == 1 and i == 3)):
        permutation = [[i - 1, i], [j - 1, j]]
        row_action = {x: [x] if x != i - 1 else [x, x + 1, x - 2] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_2_action_3_b(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if j > i + 2 and (i > 3 or (QS.s == 1 and i == 3)):
        permutation = [[i, i - 1], [j - 2, j]]
        row_action = {x: [x] if x != i - 1 else [x, x + 1, x - 2] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_2_action_3_c(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if j > i + 2 and i > 2:
        permutation = [[i, i - 1], [j - 2, j]]
        row_action = {x: [x] if x != i - 2 else [x, x + 2, x - 1] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def perm_2_2_2_action_3_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if j > i + 1 and i > 3:
        permutation = [[i, i - 1], [i - 2, i - 3], [j, j - 1]]
        row_action = {x: [x] if x != i - 1 else [x, x + 1, 0] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0
        
## row actions without permutations
def action_3_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if i >= QS.n - QS.d or QS.s <= 2:
        permutation = []
        row_action = {x: [x] if x != i else [0, x - 1, x] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def action_3_b(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if i > 2:
        permutation = []
        row_action = {x: [x] if x != i else [x - 2, x - 1, x] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def action_3_c(QS):
    j = len(QS.mat[-1]) - 1
    
    permutation = []
    row_action = {x: [x] if x != j else [x - 2, x - 1, x] for x in range(QS.n)}
    trafo = create_trafo(QS.n, permutation, row_action)
    return apply_trafo_to_QM(QS.mat, trafo)

def action_3_d(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if i > 3:
        permutation = []
        row_action = {x: [x] if x != i else [i, i - 3, i-4] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def action_4_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if i > 2:
        permutation = []
        row_action = {x: [x] if x != i else [0, x - 2, x - 1, x] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def action_2_2_a(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if j == i + 1 or j > i + 2:
        permutation = []
        row_action = {x: [x] if x != i and x != j else [x - 2, x] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)
    else:
        return 0

def action_2_2_b(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    if j - 1 > i:
        permutation = []
        row_action = {x: [x] if (x != i and x != j + 1) else [x - 1, x] if x == i else [x - 2, x] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)

def action_2_2_c(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    if j - 1 > i:
        permutation = []
        row_action = {x: [x] if (x != i and x != j) else [x - 1, x] for x in range(QS.n)}
        trafo = create_trafo(QS.n, permutation, row_action)
        return apply_trafo_to_QM(QS.mat, trafo)

def action_2_2_d(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1

    permutation = []
    row_action = {x: [x] if (x != i - 1 and x != j) else [x, x + 1] if x == i - 1 else [x - 1, x] for x in range(QS.n)}
    trafo = create_trafo(QS.n, permutation, row_action)
    return apply_trafo_to_QM(QS.mat, trafo)

# the following function creates an action_2 depending on the quaternionic structure
# mainly, this targets entries that are no powers of 2 and 
# applies calculations by which these entries are shorter products of 2 powers in the resulting matrix
def heap_of_action_2(QS):
    i = QS.number_of_complete_rows
    j = len(QS.mat[-1]) - 1
    
    new_row = QS.mat[-1]
    
    bits_new_element = get_bits(new_row[-1])

    if len(bits_new_element) > 1:
        #all two powers appearing in new element:
        two_powers = [2 ** bit for bit in bits_new_element]
        for bit in bits_new_element:
            if bit in QS.unique_two_powers_dict:
                row_number = QS.unique_two_powers_dict[bit][0]
                column_number = QS.unique_two_powers_dict[bit][-1]
                if row_number < QS.number_of_complete_rows:
                    index_limit = QS.n
                else: #row_number == QS.number_of_complete_rows
                    index_limit = j + 1
                # trafo cannot manipulate zero row if level is different from 1
                if column_number > 0 or QS.s == 1:
                    for index in range(index_limit):
                        # conditions when no trafo is possible:
                        if index == column_number or (column_number < i and index >= i) or (column_number == i and index > i) or (column_number <= j and index > j):
                            continue
                        if QS.mat[row_number][index] in two_powers:
                            row_action = {x: [x] if x != column_number else [index, x] for x in range(QS.n)}
                            trafo = create_trafo(QS.n, [], row_action)
                            if apply_trafo_to_QM(QS.mat, trafo) == 1:
                                 return 1
                if column_number <= j:
                    index_limit = i
                else:
                    index_limit = i - 1
                # trafo cannot manipulate zero row if level is different from 1
                if row_number > 0 or QS.s == 1: 
                    for index in range(index_limit):
                        # conditions when no trafo is possible (row_number and index are each at most i):
                        if index == row_number or (row_number < i and index == i):
                            continue
                        if QS.mat[index][column_number] in two_powers:
                            row_action = {x: [x] if x != row_number else [index, x] for x in range(QS.n)}
                            trafo = create_trafo(QS.n, [], row_action)
                            if apply_trafo_to_QM(QS.mat, trafo) == 1:
                                 return 1
        # always return 0 if no successful trafo has been found
        return 0
    else:
        new_element = new_row[-1]
        for row_number in range(QS.number_of_complete_rows):
            if QS.mat[row_number][j] == new_element:
                row_action = {x: [x] if x != QS.number_of_complete_rows else [row_number, x] for x in range(QS.n)}
                trafo = create_trafo(QS.n, [], row_action)
                if apply_trafo_to_QM(QS.mat, trafo) == 1:
                    return 1

        for column_number in range(j):
            if new_row[column_number] == new_element:
                row_action = {x: [x] if x != j else [column_number, x] for x in range(QS.n)}
                trafo = create_trafo(QS.n, [], row_action)
                if apply_trafo_to_QM(QS.mat, trafo) == 1:
                    return 1
        # always return 0 if no successful trafo has been found          
        return 0        


################################ 

#### FUNCTIONS TO CREATE AND APPLY BASIS TRANSFORMATIONS

def check_disjoint(test_list):
    # given a list of lists, check whether all of these are mutually disjoint 
    # this is used for assuring that a permutation is written as disjoint cycles
    return not any(set(list(sub1)).intersection(set(list(sub2)))
              for idx, sub1 in enumerate(test_list) 
              for sub2 in test_list[idx + 1:])

def create_permutation(n, permutation):
    # given a list of lists which are interpreted as disjoint cycles, permute [1, 2, ..., 2 ** (n - 1)] accordingly
    if permutation == []: # no permutation, return identity
        return [2 ** x for x in range(n)]
    if not check_disjoint(permutation): # if cycles are not disjoint, return identity and print warning
        print('permutations are not disjoint')
        return [2 ** x for x in range(n)]
    else:     
        # extend the cycles by their leading elements so that we can apply an 'index-shift'
        permutation_extended = [l + [l[0]] for l in permutation]
        affected_elements = [a for p in permutation_extended for a in p]
        return [2 ** x if x not in affected_elements else 
                2 ** affected_elements[affected_elements.index(x) + 1] for x in range(n)]

def create_trafo(n, perm, row_action):
    # given a permutation and a row action, create the transformation 'permutation followed by row action'
    permutation = create_permutation(n, perm)
    trafo = [sum([permutation[i] for i in row_action[x]]) for x in row_action.keys()]
    return trafo

def apply_trafo_to_QM(QM, trafo):
    # input: partial quaternionic matrix QM and a basis transformation 'trafo'
    # return 0 if not applicable or makes everything worse (or does not change anything, so everything stays bad)
    # return 1 if we discard the QM

    # initiate max_val (next quaternionic basis value to assign), depending on whether or not upper left value is a 1 (or a zero)
    # initiate span_new_quat (span of new quaternionic basis, ordered such that entry at index i is the value assigned to i
    match QM[0][0]:
        case 0:
            max_val = 1
            span_new_quat = [0]
        case 1:
            max_val = 2
            span_new_quat = [0, 1]
    # note that the same applies to transformed_matrix instead of QM by design

    # initiate the transformation of QM
    transformed_matrix = [[] for i in range(len(QM))]

    # row by row compute the new entries according to trafo until a better 
    # or worse entry is computed or the trafo would need an entry which is not (yet) in QM
    for i in range(len(QM)):
        bits_i = get_bits(trafo[i])
        # check whether we can access the elements with indices of trafo[i] in QM, else return 0
        if bits_i[-1] < len(QM):
            for j in range(i + 1, len(QM[i])):
                bits_j = get_bits(trafo[j])
                # check whether we can access the elements with indices of trafo[j] in QM[i], else return 0
                if bits_j[-1] < len(QM[bits_i[-1]]):
                    transformed_matrix[i].append(matrix_xor([[QM[x][y] for x in bits_i for y in bits_j]]))
                    # if the value has already been assigned, it can be found at the index of the transformed value in span_new_quat 
                    if transformed_matrix[i][-1] in span_new_quat:
                        QM_new_value = span_new_quat.index(transformed_matrix[i][-1])
                    else:
                        # else, we assign max_val as the next basis quaternion and update max_val and span_new_quat
                        QM_new_value = max_val
                        max_val = 2 * max_val
                        span_new_quat.extend([transformed_matrix[i][-1] ^ k for k in span_new_quat])
                
                    # check how the new value compares with the old one to decide whether to continue with next (i,j) or to discard the trafo or QM
                    if QM_new_value == QM[i][j]:
                        continue
                        
                    if QM_new_value > QM[i][j]:
                        return 0
                    else: # the new value must be 'better' than the one in QM
                        return 1
                else:
                    return 0
        else:
            return 0
    # if this point is reached, all entries of QM and the transformation are equal
    return 0


################################

#### USEFUL FUNCTIONS DEALING WITH THE QUATERNIONIC PRODUCT AND REPRESENTATION OF INTEGERS AS SUMS OF 2-POWERS

def quat_prod(iterab):
    # given a list iterab of integers, 
    # calculate the quaternionic product of these integers (i.e. their bitwise XOR)
    rv = 0
    for k in iterab:
        rv = rv ^ k
    return rv

def powerset(iterable):
    # given a list iterable, 
    # return a list containing all subsets of the set of list elements!
    s = list(iterable)
    return [list(x) for x in chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))]

def calculate_span(l, spn = [0]):
    # only list as input: calculate the quaternionic span of the list elements - not ordered!
    # input list and spn: update span to the span of spn union l
    span = spn
    A = set(l)
    for x in A:
        if x not in span:
            span = span + [x ^ y for y in span]
    return span

def matrix_xor(A):
    # given a matrix A, compute the bitwise XOR of all entries in this matrix
    mat = np.array(A)
    return reduce(lambda a, b: a ^ b, reduce(lambda a, b: a ^ b, mat))

def get_bits(k):
    # given a positive integer k = sum a_l 2^l, return a list which precisely contains those l with a_l = 1
    # start with an empty list
    bits = []
    for i, c in enumerate(bin(k)[:1:-1], 1):
        # if the bit at position i is one, add i - 1 to the list (index shift necessary)
        if c == '1':
            bits.append(i - 1)
    return bits

def get_integer(bits):
    # decodes a list of bits [b_1, ..., b_k] into the integer sum_{i=1}^k 2 ** b_i
    a = 0
    for i in bits:
        a = a + 2 ** i
    return a

def biggest_exp_of_two(k):
    # given a positive integer k, it returns the biggest l such that 2^l <= k
    return int(k).bit_length() - 1

# given a partial quaternionic matrix/partial completion and a new row, the following functions create/update the (implicit) completion
def create_completion(A):
    # given a partial quaternionic matrix A of size k x l, 
    # calculate the completion of A as far as possible
    k = len(A)
    l = len(A[0])
    return [[matrix_xor([[A[x][y] for y in get_bits(j)] for x in get_bits(i)]) 
             for j in range(1, 2 ** l)] for i in range(1, 2 ** k)]

def create_implicit_completion(completion):
    # given the partial completion of a quaternionic matrix A, save its implicit representation:
    # for each row, we have a dictionary, where the keys are the values occuring in this row
    # and the value is a list of the indices in which this value occurs
    return [{entry: [j for j, x in enumerate(completion[i]) if x == entry] for entry in set(completion[i])} 
              for i in range(len(completion))]
    

################################

#### THE STARTING POINT: CALCULATING POSSIBLE PARAMETERS

def create_range_of_parameters(n, s):
    # create all possible tuples (d,f,r,v,t,u) of parameters 
    # for nonrigid, nondegenerate and nonpythagorean normal quaternionic matrices with level s
        
    # d (for s <= 2) or d + 1 (for s > 2) is the number of nonzero entries in the first row  
    # which can range from 1 to n - 2, excluding the degenerate, pythagorean and rigid cases
    # for s <= 2, we can also exclude 2-rigid cases, the case d = n - 2 can be dropped
    if s <= 2:
        d_range = range(1, n - 2)
    else:
        d_range = range(n - 2)

    # the second block is multiplied by factor f which is always 1 for s <= 2 or d = 0 and can be chosen from {1,2} else
    d_f_range = [[d] + [f] for d in d_range 
                     for f in range(1, min((s + 1).bit_length(), d + 2))]

    # the length of the block 2 is r, it can be at most n-d-2 as block 1 has length at least 2 and the first three blocks have length n-d
    # r can be at most d (if s <= 2 or f = 2) or d + 1 (if s = 4 and f = 1) because in the 2nd row of block 2, nonzero values from the 1st row appear
    # if f = 2, then r has to be at least 1 so that there is a block to multiply by 2
    if s <= 2: # in this case, always f = 1, so the lower bound on r is zero
        d_f_r_range = [x + [r] for x in d_f_range 
                    for r in range(0, min(n - x[0] - 1, x[0] + 1))]     
    else:
        d_f_r_range = [x + [r] for x in d_f_range 
                    for r in range(x[1] - 1, min(n - x[0] - 1, x[0] + 3 - x[1]))]
        # the following excludes parameters which produce a 2-rigid first row and level exactly 4, 
        # as these do not have to be considered due to the result of Cordes
        # the case to be excluded is d = n - 3 (first row two zeros) with f = 1, r = 1 (there is entry 1 in block 2) 
        # note that d = n - 3 implies r <= 1 in all cases, so it suffices to discern whether or not we have r == 0 here
        d_f_r_range = [x for x in d_f_r_range if (x[0] != n - 3 or x[1] == 2 or x[2] == 0)] 

    # since block 1 has length n - r - v >= 2, we get v <= n - r - 2
    # since block 3 has length v - d >= 0, we get v >= d
    # if d = 0 and r = 0, we can, excluding degenerate cases, assume v >= 1 (else, we would get length of block 1 as n - r - v = n)
    d_f_r_v_range = [x + [v] for x in d_f_r_range 
                      for v in range(max(x[0], 1 - x[2]), n - x[2] - 1)]

    # we always have t <= d (comparing length of block 4 with length of blocks 4 to 6)
    # if s = 1, then t <= r + v - d ( >= r ) since there cannot be more zeros in the 2nd row than in the 1st
    # if s = 2 or f = 2, then t >= r as block 4 has to be of at least the size of block 2 because all values in the second row of block 2 appear in the first row of block 4
    # if s = 1 or s = 2, we also have t >= 1 if v + r = n - 2 is maximal, since we exclude 2-rigid cases, this translates to t >= 3 - n + r + v
    # if s = 4 and f = 1, then t >= max(0, r - 1) (cf. previous case, but one of the values in block 2 ('1') does not appear in block 4)
    # excluding degenerate cases, the length of block 1 + block 4 can be at most n - 1, so t <= r + v - 1
    match s:
                case 1:
                    d_f_r_v_t_range = [x + [t] for x in d_f_r_v_range 
                                       for t in range(max(x[2], 3 - n + x[2] + x[3]), min(x[0] + 1, x[2] + x[3] - x[0] + 1))]
                case 2:
                    d_f_r_v_t_range = [x + [t] for x in d_f_r_v_range 
                                       for t in range(max(x[2], 3 - n + x[2] + x[3]), min(x[0] + 1, x[2] + x[3]))]
                case 4:
                    d_f_r_v_t_range = [x + [t] for x in d_f_r_v_range 
                                       for t in range(max(0, x[2] + x[1] - 2), min(x[0] + 1, x[2] + x[3]))]


    # by the sizes of blocks 5 and 6, we have t <= u <= d
    # if s <= 2, block 5 can't be bigger than block 4 
    # else replace 2nd basis element by product with 1st and exclude factors from block 2, giving u <= 2t-r (cf. article)
    if s <= 2:
        d_f_r_v_t_u_range = [x + [u] for x in d_f_r_v_t_range
                         for u in range(x[4], min(2 * x[4] - x[2], x[0]) + 1)]
    else:
        d_f_r_v_t_u_range = [x + [u] for x in d_f_r_v_t_range
                         for u in range(x[4], x[0] + 1)]

    # to exclude 2-rigid (for s <= 2) or rigid (for s = 4) third row in the completion, we remove the case t = u and v = n - 2 
    d_f_r_v_t_u_range = [x for x in d_f_r_v_t_u_range if (x[3] != n - 2 or x[4] != x[5])]
    
    return d_f_r_v_t_u_range


################################

####  THE CLASS OF QUATERNIONIC STRUCTURES

class quaternionic_structure():

    ## VALIDITY CHECKS
    # for every new row, check whether the corresponding updated implicit completion is nondegenerate, nonrigid, and satisfies the common slot property
    def check_common_slot_implicit(self, row_to_check):
        # given a quaternionic structure and an index row_to_check, 
        # compare this row of the completion with previous full rows regarding common slots
        # this function is called whenever the implicit completion is extended

        # collect quaternions in the row to check, remove zero as the common slot property is always fulfilled for zero
        values_in_row_to_check = set(self.completion[row_to_check].keys())
        values_in_row_to_check.remove(0)

        # completion at row to check has length determined by last row of the matrix
        length_completion_row_to_check = 2 ** len(self.mat[-1]) - 1
        
        # we can only compare with full rows up to row_to_check in the completion if we want to find violations of the common slot property
        # if the last row of self.mat (associated with index 2^number_of_complete_rows in the completion) is not yet complete, only the previous rows are complete
        max_index = 2 ** self.number_of_complete_rows - 1
        # if the last row is complete, row_to_check can be compared with all previous rows
        # however, tests imply that this has a negative effect on run times without finding more violations to common slot
        # hence, the following two lines of code are currently not used
        ####if len(self.mat[-1]) == self.n:
        ####    max_index = row_to_check

        for i in range(max_index):
            # collect the relevant quaternion algebras of the lines under consideration                
            # since we update the completion '2-power interval by 2-power interval', so even if value[-1] is not necessarily the maximum index,
            # it uniquely determines the maximal 'interval' where a key appears in row i of the completion
            # hence, we can check for common slot if the value doesn't appear in the intervals that are not yet defined for row_to_check
            values_in_line_i_partial = set(key for key, value in self.completion[i].items() 
                                           if value[-1] < length_completion_row_to_check)
    
            # calculate which quaternion algebras occur both in line i and row_to_check
            # 0 does not have to be checked since clearly, we have 0 = q(_, 1) everywhere, so common slot is always fulfilled for zero
            common_quaternion_values = (values_in_line_i_partial & values_in_row_to_check)
    
            # if there are nonzero quaternions appearing in both lines, we can compare whether they appear at a common index
            if common_quaternion_values:
                for q in common_quaternion_values:
                    # for such q, there are x, y such that (i, x) = q = (j, y).
                    # due to the common slot property, there has to be a z such that (i, x) = (i, z) = (j, z) = (j, y)
                    # else, we cannot extend this structure to a full quaternionic structure
                    if set(self.completion[i][q]).isdisjoint(set(self.completion[row_to_check][q])):
                        return False
                        
        # if we come to the following line, the common slot property is fulfilled everywhere where we can check up to this point
        return True

    def check_degenerate_and_rigid_implicit(self, row_to_check):
        # input: quaternionic structure and index row_to_check of a newly extended row of the completion
        # check whether the row in the completion indexed 'row_to_check' is degenerate or ' too rigid'
        # the latter means that row to check is 0-rigid, 1-rigid or -in all cases where this can be excluded via theoretical results- 2-rigid
        # for s = 1 also check whether row_to_check is 'less rigid' than the first row, 
        # meaning we could apply a basis transformation to exclude the current case
        if self.s == 1 or (self.s == 2 and 0 in self.completion[row_to_check][0]):
            rigmin = 2 # these are the cases when we can exclude '2-rigid'
        else:
            rigmin = 1 # we can always exclude '1-rigid'
        # check whether the current row has enough entries to say anything definite about rigidity
        if len(self.mat[-1]) > self.n - rigmin:
            # degenerate: only 0s in a row, this is equivalent to there being more than 2^(n-1) zeros in a row
            # k-rigid: 2^k - 1 zeroes in a row
            number_of_zeros = len(self.completion[row_to_check][0])
            if (number_of_zeros <= (2 ** (rigmin + len(self.mat[-1]) - self.n) - 1)) or (self.s == 1 and number_of_zeros > (2 ** (self.n - self.d))) or (self.s > 1 and number_of_zeros > 2 ** (self.n - 1)):
                return False
        # return true if no violations have been found
        return True

    def check_basis_trafo(self):
        # given a partial quaternionic structure, check all basis transformations from a predefined list
        # return False if we can discard the QS by one of the selected trafos and True otherwise

        # selected transformations:
        all_trafos = [heap_of_action_2, # doing heap first seems to slightly enhance performance
              perm_2_a, perm_2_b, perm_2_c,
              action_3_a, action_3_b, action_3_c,action_3_d,
              action_4_a,
              action_2_2_a, action_2_2_b, action_2_2_c, action_2_2_d, 
              perm_2_2_a, perm_2_2_b, perm_2_2_c, perm_2_2_d,
              perm_2_action_2_a, perm_2_action_2_b, perm_2_action_2_c, perm_2_action_2_d, perm_2_action_2_e, perm_2_action_2_f,
              perm_2_action_3_a,
              perm_2_2_action_2_a, perm_2_2_action_2_b,
              perm_2_2_action_3_a, perm_2_2_action_3_b, perm_2_2_action_3_c, 
              perm_2_2_2_action_3_a,
              perm_3_2_a, perm_4_2_a, perm_4_3_a, perm_5_2_a]
        # check all trafos until there is one that improves the quaternionic matrix
        for trafo in all_trafos:
            if trafo(self) == 1:
                return False
        # return true if no violations have been found
        return True

    def compare_first_three_blocks(self):
        # given a quaternionic structure where the newly extended row has length self.n - self.d and starts with a zero,
        # compare whether choosing the current row as second row would have created a 'better' set of parameters, i.e. 
        # lexicographically smaller first two rows up to entry [1][self.n - self.d]
        # return 0 iff we can discard new_row
        # return 1 iff bad second row: continue to complete new row
        # return 2 iff should be further checked in the sequel
        
        if self.mat[-1][0] != 0:
            # second row has to start with zero
            return 1
        else:
            # check the entries under the 0s of the first row
            # calculate the dimension of span of entries of new row, compare to the dimension of second row ( = r + v - d)
            span_of_new_row = set(calculate_span(self.mat[-1]))

            if len(span_of_new_row) == 2 ** (self.r + self.v - self.d):
                # check further
                if self.s <= 2:
                    # the dimension of the intersection of span_new_row with the span of the first row determines the parameter r
                    dim_intersection = biggest_exp_of_two(len([x for x in span_of_new_row if x < 2 ** self.d]))
                    if dim_intersection == self.r:
                        return 2
                    elif dim_intersection < self.r:
                        return 1
                    else:
                        return 0
                else:
                    # check whether the 'new block 2' would have a lexicographically smaller beginning
                    if self.f == 2 or self.r == 0:
                        if 1 in span_of_new_row:
                            return 0
                        else:
                            # same check for parameter r as before
                            dim_intersection = biggest_exp_of_two(len([x for x in span_of_new_row if x < 2 ** (self.d + 1)]))
                            if dim_intersection == self.r:
                                return 2
                            elif dim_intersection < self.r:
                                return 1
                            else:
                                return 0
                    else: # in this case, there is the entry '1' in the current second row
                        if 1 in span_of_new_row:
                            dim_intersection = biggest_exp_of_two(len([x for x in span_of_new_row if x < 2 ** (self.d + 1)]))
                            if dim_intersection == self.r:
                                return 2
                            elif dim_intersection < self.r:
                                return 1
                            else:
                                return 0
                        else:
                            return 1
            # if the dimension of the current row is smaller than that of the 2nd row (first three blocks), we can use it as a better 2nd row
            # note that we only check this once we have reached the index of the last zero in the first row, so we compare the entire blocks
            elif len(span_of_new_row) < 2 ** (self.r + self.v - self.d):
                return 0
            else:
                return 1

    def compare_last_three_blocks(self):
        # input: partial quaternionic structure where the last row has been extended to length n and the first_three_blocks check for the
        # last row had the result 'produces the same first three blocks as the second row' (i.e. output '2')
        # return False if the new row does provide a better second row, i.e. the partial quaternionic structure can be discarded
        # return True if there is no violation found

        # calculate the span and dimension of the new row
        span_new_row = set(calculate_span(self.mat[-1]))
        dim_new_row = biggest_exp_of_two(len(span_new_row)) # = r + v - t if that was the second row
        # if dim_new_row is smaller than the dimension of the second row, it provides more leading zeros, i.e. a lexicographically better choice
        # note that this only works because the dimension regarding the first three blocks is equal by our previous checks
        if dim_new_row > self.r + self.v - self.t:
            return True
        if dim_new_row < self.r + self.v - self.t:
            return False

        # now we know the equality of the t-values since the r- and v-values are equal by the first_three_blocks check
        # we thus compare the u-values
        # r + u - t = length block 2 + block 5
        # note that self.mat[0][-1] cannot be zero as at least one of the last three blocks appears
        # block 2 and block 5 consist of those values that can already be found in the first row of the completion
        dim = biggest_exp_of_two(len(set([x for x in span_new_row if x < 2 * self.mat[0][-1]])))
        if dim > self.r + self.u - self.t:
            return False
        else:
            return True
        
        
    ## INITIATING AND EXTENDING THE STRUCTURE
    def create_first_and_second_row(self):
        # given a set of parameters, construct the first two rows
        # of the quaternionic matrix with these parameters
    
        # block structure for s <= 2:
        #    1    |        2      |         3       |       4       |        5        |        6
        # 0 ... 0 | 0 ......... 0 | 0 ........... 0 | 1 ... 2^{t-1} | 2^t ... 2^{u-1} | 2^u ... 2^{d-1}
        # 0 ... 0 | 1 ... 2^{r-1} | 2^d ... 2^{v-1} | 0 ......... 0 | 2^t ... 2^{u-1} | 2^v ... 2^{v+d-1-u}
        #  n-r-v  |        r      |        v-d      |        t      |       u-t       |      d-u
    
        # block structure for s = 4: - block 2 may be multiplied by 2
        #    1      |        2      |         3       |     4     |        5        |        6
        # 1 0 ... 0 | 0 ......... 0 | 0 ........... 0 | 2 ... 2^t | 2^{t+1} ... 2^u | 2^{u+1} ... 2^d
        # 0 0 ... 0 | 1 ... 2^{r-1} | 2^{d+1} ... 2^v | 0 ..... 0 | 2^{t+1} ... 2^u | 2^{v+1} ... 2^{v+d-u}
        #  n-r-v    |        r      |       v-d       |     t     |       u-t       |      d-u
    
        # creates a zero block
        block_1 = np.zeros((2, self.n - self.r - self.v), dtype = int)
        
        if self.s <= 2:
            factor = 1
        else:
            # for s = 4, the matrix starts with a 1
            # further, blocks 3 to 6 have to be multiplied by 2
            factor = 2
            block_1[0, 0] = 1
    
        # creates block of form (2 x r), first row zeros, second row powers of 2 from 1 to 2^{r-1}, multiplied by 2 if factor_block_2 == 2 
        block_2 = self.f * np.array([[0 if i == 0 else 2 ** j for j in range(self.r)] 
                            for i in range(2)])
    
        # creates block of form (2 x (v - d)), first row zeros, second row powers of 2 from 2^d to 2^{v-1}, multiplied by factor
        block_3 = factor * np.array([[0 if i == 0 else 2 ** (self.d + j) for j in range(self.v - self.d)] 
                            for i in range(2)])
    
        # creates block of form (2 x t), first row powers of 2 from 1 to 2^{t-1}, second row zeros, multiplied by factor
        block_4 = factor * np.array([[2 ** j if i == 0 else 0 
                    for j in range(self.t)] for i in range(2)])
    
        # creates block of form (2 x (u - t)), first and second row powers of 2 from 2^t to 2^{u-1}, multiplied by factor
        block_5 = factor * np.array([[2 ** (j + self.t) 
                    for j in range(self.u - self.t)] for i in range(2)])
    
        # creates block of form (2 x (d - u)), first row powers of 2 from 2^u to 2^{d-1}, second row from 2^v to 2^{v+d-u-1}, multiplied by factor
        block_6 = factor * np.array([[2 ** (j + self.u) if i == 0 else 2 ** (j + self.v)
                    for j in range(self.d - self.u)] for i in range(2)])
    
        return (np.concatenate((block_1, block_2, block_3, 
                    block_4, block_5, block_6), axis = 1, dtype = int, casting = 'unsafe')).tolist()

    def add_new_row(self):
    # this function adds the beginning of the next row to a partial quaternionic matrix, taking into account symmetry and the diagonal rule M3    
        # diagonal entry is 0 for level 1 and the same as the first row entry (same column) else
        if self.s == 1:
            diag = 0
        else:
            diag = self.mat[0][self.number_of_complete_rows]

        # the beginning of new row can be determined by symmetry 
        new_row = [self.mat[i][self.number_of_complete_rows] if i < self.number_of_complete_rows 
                   else diag for i in range(self.number_of_complete_rows + 1)]

        # create new QS adding new_row and checking rigidity and common slot but no trafos since we only added up to the diagonal
        new_QS = quaternionic_structure(QS = self, extension = new_row)

        # check whether the new structure is valid and 
        # update whether or not swapping the current row with the previous one is still a possible transformation
        if new_QS and new_QS.validity:
            if self.number_of_complete_rows < self.n - 1:
                # update whether previous row check - can the current row still be swapped with the previous one?
                if new_QS.do_previous_row_check:
                    for k in range(len(new_QS.mat[-1]) - 2):
                        if new_QS.mat[-1][k] != 0 and new_QS.mat[-2][k] == 0:
                            new_QS.do_previous_row_check = False
                            break
            return new_QS
        else:
            del new_QS
            return None
    
    def calc_possible_next_elements(self):
        # given a QS, this function calculates the set of possible values by which an incomplete row of a quaternionic matrix can be extended
        # it takes into account some basis transformations (implicitly) and some consequences of the common slot property and rigidity
        new_row = self.mat[-1]
        length_new_row = len(new_row)

        # check whether there is still an entry to add to new_row
        if length_new_row < self.n:   
            # check for simple transformations replacing the current row or column by a product with another
            # if the product is possible, we can exclude certain factors for the next entry (cf. paper, end of chapter 5)
            pre_excl_list = []
            exclusion_list = []
            two_power_list = []
            # in the current row, check which entries are not unique (unique values will not be changed by the considered transformations)           
            relevant_indices_row = [i for i in range(length_new_row) if 
                                (i != self.number_of_complete_rows and biggest_exp_of_two(self.mat[-1][i]) not in self.unique_two_powers_dict)]
            # check whether previous rows provide new factors to exclude (i.e. zeros at the relevant indices)
            for i in range(self.number_of_complete_rows):
                if self.mat[i][length_new_row] != 0:
                    if not any([self.mat[i][j] for j in relevant_indices_row]):
                        bits = get_bits(self.mat[i][length_new_row])
                        if len(bits) == 1: # in this case, we can exclude a power of 2 straightforward
                            exclusion_list.append(self.mat[i][length_new_row])
                            two_power_list.append(bits)
                        else: # in this case, we can possible exclude some of the 2 powers, depending on the other entries of the exclusion list
                            pre_excl_list.append(bits)

            # do the same for columns
            relevant_indices_column = [i for i in range(self.number_of_complete_rows) if 
                                       biggest_exp_of_two(self.mat[i][length_new_row]) not in self.unique_two_powers_dict]
            for j in range(length_new_row):
                if self.mat[-1][j] != 0:
                    if not any([self.mat[i][j] for i in relevant_indices_column]):
                        bits = get_bits(self.mat[-1][j])
                        if len(bits) == 1:
                            exclusion_list.append(self.mat[-1][j])
                            two_power_list.append(bits)
                        else:
                            pre_excl_list.append(bits)

            # in the pre_excl_list, we clear out all factors that can be excluded anyway, then we update to integers
            pre_excl_list_update = [[x for x in A if x not in two_power_list] for A in pre_excl_list]
            pre_excl_list_numbers = [get_integer(A) for A in pre_excl_list_update]

            # we can also exclude the biggest powers of 2 that appear in (products of) elements from the pre excl list
            exclusion_list.extend([2 ** biggest_exp_of_two(quat_prod(x)) for x in powerset(pre_excl_list_numbers)])

            new_2_power = 2 ** (self.max_exp_of_two + 1)

            possible_basis_elements_current_entry = set(self.possible_basis_elements).difference(exclusion_list)
            # the possible values are now all products of the not excluded factors, or the new 2 power
            
            possible_values = calculate_span(possible_basis_elements_current_entry) + [new_2_power]

            # early common slot check to avoid some computations
            # check whether a value is in the span of the new row and only in the interval of a row in the completion defined by the next entry
            # then, the next entry has to be in span_new_row
            span_new_row = [*self.completion[2 ** (len(self.mat) - 1) - 1]]
            for i in range(2 ** (len(self.mat) - 1) - 1):
                for a in span_new_row:
                    if a in self.completion[i]:
                        # in principle, we had to check min and max of the completion, but by construction, it is divided into partions
                        # of intervals between 2-powers shifted by 1
                        # any elements only appearing in one interval need to appear in this interval whenever they appear in a row of the completion
                        if (self.completion[i][a][0] > 2 ** length_new_row - 2) and (self.completion[i][a][-1] < 2 ** (length_new_row + 1) - 1):
                            possible_values = [x for x in possible_values if x in span_new_row]
                            break
                else:
                    # this else - break construction assures that once we break out of the inner loop, we also break out of the outer one
                    continue
                break

            # if we can swap with the previous row, we cannot have a smaller entry in the current row
            # if we can replace by product with the previous row and the previous row has only unique elements from current entry on,
            # we cannot have any entry from the span of the previous row (consider product plus common slot to create a smaller previous row)
            if self.do_previous_row_check:
                if self.mat[-1][-1] != 0 and self.mat[-2][length_new_row - 1] == 0:
                    self.do_previous_row_check = False
                elif self.mat[-2][length_new_row] != 0:
                    min_element_prev_row = min([x for x in self.mat[-2] if x > 0])
                    # exclude those values that lie in the span of the union of current row and previous row - span of previous row is the second argument
                    possible_values = [x for x in possible_values if (x not in calculate_span(self.mat[-1], [*self.completion[2 ** (len(self.mat) - 2) - 1]])) and (x > min_element_prev_row)]

            # do the same for columns
            if self.mat[-1][-1] != 0 and length_new_row > len(self.mat):
                do_previous_column_check = True
                for i in range(len(self.mat) - 1):
                    if (self.mat[i][length_new_row] != 0) and (biggest_exp_of_two(self.mat[i][length_new_row - 1]) not in self.unique_two_powers_dict):
                        do_previous_column_check = False
                        break   
                if do_previous_column_check:
                    possible_values = [x for x in possible_values if x >= self.mat[-1][-1]]
         
            # precheck column for rigidity
            if self.s == 1 or (self.s == 2 and 0 == self.mat[0][length_new_row - 1]):
                rigmin = 2
            else:
                rigmin = 1
            if self.number_of_complete_rows >= self.n - rigmin - 1:
                span_column = calculate_span([self.mat[k][length_new_row] for k in range(self.n - rigmin - 1)])
                if len(span_column) == 2 ** (self.n - rigmin - 1):
                    possible_values = [x for x in possible_values if x in span_column]

            # precheck row for rigidity
            if self.s == 1 or (self.s == 2 and 0 == self.mat[-1][0]):
                rigmin = 2
            else:
                rigmin = 1
            if len(span_new_row) >= 2 ** (self.n - rigmin - 1):
                possible_values = [x for x in possible_values if x in span_new_row]

            # create extended matrices   
            return possible_values
        else:   
            return []
    
    def create_completion(self):
        # computes the completion of self.mat, needed to initiate the implicit completion inside the class of quaternionic structures
        return create_completion(self.mat)

    def update_implicit_completion(self):
        # update the implicit completion for the new entry of the current row

        new_row = self.mat[-1]
        length_new_row = len(new_row)
        length_completion_full_rows = 2 ** self.number_of_complete_rows - 1

        # if we just added a new row, we do not have any previous data 
        if length_new_row == self.number_of_complete_rows + 1:
            new_row_explicit_completion_list_of_list = create_completion([new_row])
            new_row_implicit_completion = create_implicit_completion(new_row_explicit_completion_list_of_list)
            new_row_explicit_completion = new_row_explicit_completion_list_of_list[0]
            self.completion.append(*new_row_implicit_completion)
        else:
            # compute the explicit completion in the new bounds 
            new_row_explicit_completion = [self.mat[-1][-1] if i == 0 else 0 for i in range(2 ** (length_new_row - 1))]

            # update the implicit completion of new row by considering the xor's of the already computed part with the new entry
            # key: entries of the already computed part of the completion row
            # value: list of indices, where key appears
            dict_to_append = {}
            for key, value in self.completion[length_completion_full_rows].items():
                for v in value:
                    new_key = self.mat[-1][-1] ^ key
                    # update explicit version for later calculations
                    new_row_explicit_completion[v + 1] = new_key
                    
                    # update implicit version
                    # in the updated completion, new_key appears as the xor of the last computed entry of self.mat and key at index new_value                   
                    new_value = v + 2 ** (length_new_row - 1)
                    # as before, update the dictionary with regard to the maximal index being last
                    if new_key in dict_to_append:
                        dict_to_append[new_key].append(new_value)
                    else:
                        dict_to_append.update({new_key: [new_value]})

            # update the implicit completion of the new row by the new entry of self.mat
            index_diag = 2 ** (len(self.mat[-1]) - 1) - 1
            if self.mat[-1][-1] in self.completion[length_completion_full_rows]:
                self.completion[length_completion_full_rows][self.mat[-1][-1]].append(index_diag)
            else:
                self.completion[length_completion_full_rows].update({self.mat[-1][-1]: [index_diag]})
                
            # append further entries
            for key in dict_to_append:
                if key in self.completion[length_completion_full_rows]:
                    self.completion[length_completion_full_rows][key].extend(dict_to_append[key])
                else:
                    self.completion[length_completion_full_rows].update({key: dict_to_append[key]})

            new_row_implicit_completion = self.completion[length_completion_full_rows]

        # here, the row of the completion which is associated directly with the laast row of the ucrrent matrix, has been updated
        # do validity checks
        if not self.check_degenerate_and_rigid_implicit(length_completion_full_rows):
            self.validity = False
            return                        
        if not self.check_common_slot_implicit(length_completion_full_rows):
            self.validity = False
            return
        
        # now, update the further rows that come from products of the new row with previous ones
        for i in range(length_completion_full_rows):
            # take care of index shift
            current_row_in_completion = i + length_completion_full_rows + 1

            # again, if we just added a new row and not only appended an element, we have to initialize
            # min_index and max_index indicate the indices for which we update
            if length_new_row == self.number_of_complete_rows + 1:
                self.completion.append(dict())
                min_index = 0
            else:
                min_index = 2 ** (length_new_row - 1) - 1
            max_index = 2 ** length_new_row - 2

            for key, value in self.completion[i].items():
                for index in value:
                    if index in range(min_index, max_index + 1):
                        if min_index == 0:
                            index_shift = 0
                        else:
                            index_shift = 2 ** (length_new_row - 1) - 1

                        # update similarly to the above
                        new_key = new_row_explicit_completion[index - index_shift] ^ key

                        if new_key in self.completion[current_row_in_completion]:
                            self.completion[current_row_in_completion][new_key].append(index)
                        else:
                            self.completion[current_row_in_completion].update({new_key: [index]})

            # do validity checks
            if not self.check_degenerate_and_rigid_implicit(current_row_in_completion):
                self.validity = False
                return                        
            if not self.check_common_slot_implicit(current_row_in_completion):
                self.validity = False
                return

    def downgrade_completion(self, deleted_row):
        # given a QS and a boolean value stating whether or not we deleted the last row (True) or only the last entry of the last row (False),
        # delete all associated values in the completion
        
        number_of_rows = len(self.mat)
        # delete all rows that are no products of the remaining rows of the matrix
        if deleted_row:
            del self.completion[2 ** number_of_rows - 1:]

        # from the following index on, the entries in the completion associated with any products involving the last row of the matrix are not defined
        critical_index = 2 ** len(self.mat[-1]) - 1

        # for the rows in the completion associated to products involving the last row of self.mat, collect the keys to delete 
        # and delete all indices from critical_index on in the keys that are not entirely deleted
        # this makes use of the fact that the values of each key are sorted by 2-power intervals (shifted by index 1)
        for i in range(2 ** (number_of_rows - 1) - 1, len(self.completion)):
            keys_to_delete = []
            
            for key in self.completion[i]:
                if self.completion[i][key][0] >= critical_index:
                    keys_to_delete.append(key)
                else:
                    while self.completion[i][key][-1] >= critical_index:
                        del self.completion[i][key][-1]
            for key in keys_to_delete:
                del self.completion[i][key]
        
    def update_unique_two_powers_dict(self):
    # update unique 2-powers dictionary of a given quaternionic structure
        # if the new row is only defined up to the diagonal, there are no changes to the unique 2 powers
        if len(self.mat[-1]) != self.number_of_complete_rows + 1:
            # consider the maximal appearing power of 2 before the newly computed entry
            # check whether new entry is a new unique 2 power (i.e. bits==[max_bit]) and update the dictionary
            # else, delete all entries from the dictionary that appear as a factor in the new entry as they are not unique anymore
            bits = get_bits(self.mat[-1][-1])
            if len(bits) == 1 and bits[0] > self.max_exp_of_two:
                self.unique_two_powers_dict.update({bits[0]: [len(self.mat) - 1, len(self.mat[-1]) - 1]})
                self.max_exp_of_two = self.max_exp_of_two + 1
                self.possible_basis_elements.append(self.mat[-1][-1])
            else:
                self.unique_two_powers_dict = {x: self.unique_two_powers_dict[x] for x in self.unique_two_powers_dict if x not in bits}


    ## OUTPUT AND DEALING WITH FILES, AUXILIARY FUNCTIONS
    def print_quaternionic_matrix(self, print_completion = False):
        # this prints quaternionic matrices in a row by row format
        # optionally also print the completion
        for row in self.mat:
            print(row)

        if print_completion:
            print('Corresponding Completion: ')
            for row in self.completion:
                print(row)

    def get_parameters(self):
        # function for returning the parameters s,d,f,v,t,u as a list
        return [self.s, self.d, self.f, self.r, self.v, self.t, self.u] 


    ## INITIATING AND UPDATING AN OBJECT 'QUATERNIONIC STRUCTURE'
    def __init__(self, n = None, s = None, d = None, f = None, r = None, v = None, t = None, u = None, QS = None, extension = None):
        if not QS:
            self.validity = True # update to false whenever the structure can be discarded for any reason
            # parameters:
            self.n = n
            self.s = s
            self.d = d
            self.f = f
            self.r = r
            self.v = v
            self.t = t
            self.u = u
            # initiate first two rows and properties
            self.continue_second_row_checks = True
            self.mat = self.create_first_and_second_row()
            self.number_of_complete_rows = 2
            self.completion = create_implicit_completion(self.create_completion())
            # check validity of the completion
            for i in range(3):
                if not self.check_degenerate_and_rigid_implicit(i):
                    self.validity = False
                    return
                else:
                    if not self.check_common_slot_implicit(i):
                        self.validity = False
                        return

            # initiate the dictionary of unique 2 powers
            if self.s == 4:
                two_power_block_one = {0: [0]}
                correction_term = 1
            else:
                two_power_block_one = {}
                correction_term = 0

            # initiate max_exp_of_two appearing in self.mat (must appear at ending of a row or at ending of block 3 for second row)
            # initiate possible basis elements for entries in matrix
            self.max_exp_of_two = biggest_exp_of_two(max([self.mat[0][-1], self.mat[1][-1], self.mat[1][self.n - self.d - 1]]))
            self.possible_basis_elements = [2 ** i for i in range(self.max_exp_of_two + 1)]
            # the unique 2 powers are those that appear in block 1 (if s=4), and blocks 3,4,6 but not in block 2
            two_powers_first_row_blocks_one_four = two_power_block_one | {biggest_exp_of_two(x): 
                                                                          [0, self.n - self.d + biggest_exp_of_two(x) - correction_term] 
                                                                          for x in self.mat[0][self.n - self.d:self.n - self.d + self.t] if x not in self.mat[1]}

            two_powers_first_row_blocks_one_four_six = two_powers_first_row_blocks_one_four | {biggest_exp_of_two(x): 
                                                                                                [0, self.n - self.d + self.u + biggest_exp_of_two(x) - self.u - correction_term] 
                                                                                               for x in self.mat[0][self.n - (self.d - self.u):]}
                
            two_powers_second_row_blocks_three = {biggest_exp_of_two(x): [1, self.n - self.v + biggest_exp_of_two(x) - self.d - correction_term] 
                                                  for x in self.mat[1][self.n - self.v:self.n - self.d]}
                                                          
            two_powers_second_row_blocks_three_six = two_powers_second_row_blocks_three | {biggest_exp_of_two(x): 
                                                                                           [1, self.n - self.d + self.u + biggest_exp_of_two(x) - self.v - correction_term] 
                                                                                            for x in self.mat[1][self.n - (self.d - self.u):]}
                                                          
            self.unique_two_powers_dict = two_powers_first_row_blocks_one_four_six | two_powers_second_row_blocks_three_six
            # if the second row has the zeros first and the nonzero entries last, we can do the previous row check in the third row
            if self.n == self.d or self.t == 0:
                self.do_previous_row_check = True
            else:
                self.do_previous_row_check = False
        else:
            # copy the basic properties
            self.validity = True
            self.n = QS.n
            self.s = QS.s
            self.d = QS.d
            self.f = QS.f
            self.r = QS.r
            self.v = QS.v
            self.t = QS.t
            self.u = QS.u
            self.mat = QS.mat
            if type(extension) == int:
                self.mat[-1].append(extension)
            else:
                self.mat.append(extension)
            self.completion = QS.completion
            self.number_of_complete_rows = len(self.mat) - 1
            self.continue_second_row_checks = QS.continue_second_row_checks
            self.do_previous_row_check = QS.do_previous_row_check

            # legend for check_ftb:
            # return 0 iff we can discard new_row
            # return 1 iff bad second row: continue to complete new row, no further checks needed
            # return 2 iff should be further checked in the sequel
            
            # compare first three blocks with second row if that part of the current row is done  
            if len(self.mat[-1]) == self.n - self.d and self.continue_second_row_checks:
                check_ftb = self.compare_first_three_blocks()
                match check_ftb:
                    case 0:
                        self.validity = False
                        return
                    case 1:
                        self.continue_second_row_checks = False
                    case 2:
                        pass 
            
            # check whether the updated completion still is nondegenerate and nonrigid
            # whenever validity is turned to false, discard the structure
            if len(self.mat[-1]) == QS.n:
                if self.continue_second_row_checks and self.d > 0:
                    self.validity = self.compare_last_three_blocks()

            if self.validity == False:
                return
                
            self.update_implicit_completion()
            if self.validity == False:
                return
        
            self.unique_two_powers_dict = copy.deepcopy(QS.unique_two_powers_dict)

            # we only do basis transformations if the last row has entries beyond the diagonal
            if len(self.mat[-1]) > self.number_of_complete_rows + 1:
                if not self.check_basis_trafo():
                    self.validity = False
                    return

            # update max_exp_of_two and possible_basis_elements
            self.max_exp_of_two = QS.max_exp_of_two
            self.possible_basis_elements = copy.deepcopy(QS.possible_basis_elements)
            
            # update unique_two_power_list
            self.update_unique_two_powers_dict()

            # once the new row is completed, check whether second row checks are still possible and
            # check whether a previous row check is possible
            if len(self.mat[-1]) == QS.n:
                self.number_of_complete_rows = len(self.mat)
                if self.number_of_complete_rows < QS.n:
                    if self.mat[0][self.number_of_complete_rows] == 0:
                        self.continue_second_row_checks = True
                    else:
                        self.continue_second_row_checks = False
                    
                    # check if we can use the just finished row for previous row checks
                    dim_span_new_row = self.n - biggest_exp_of_two(len(self.completion[2 ** (len(self.mat) - 1) - 1][0]) + 1)

                    if not any(self.mat[-1][:self.n - dim_span_new_row]):
                        self.do_previous_row_check = True
                    else:
                        self.do_previous_row_check = False


################################

#### FUNCTIONS FOR PRINTING AND SAVING RESULTS

def print_quaternionic_matrices_dict(QM_dict, print_explicit_matrices = True, print_completion = False, print_basis_trafos = False):
    # this function prints the matrices of a given dictionary QM_dict of quaternionic structures
    if len(QM_dict.keys()) > 1:
        # if multiple levels are considered, print the number of all structures first
        print('Number of all structures: ' + str(sum([len(QM_dict[s]) for s in QM_dict.keys()])))

    # for every level, print the number of structures of this level
    for s in QM_dict.keys():
        if s != 4:
            print('Level: ' + str(s))
        else:
            print('Level: >= ' + str(s))
        print('Number of structures: ' + str(len(QM_dict[s])))
        # optionally print all matrices explicitly
        if print_explicit_matrices:
            for quat_struct in QM_dict[s]:
                print('Quaternionic Matrix:')
                print_qm(quat_struct)


def print_quaternionic_matrices_with_parameters(QM_dict, parameters, print_explicit_matrices = True):
    # this function prints all matrices of a given dictionary of quaternionic structures along with their parameters
    
    list_of_QM_with_parameters = [QM for QM in QM_dict[parameters[0]] if QM.get_parameters() == parameters]
    if len(list_of_QM_with_parameters) > 0:
        print('Number of all structures: ' + str(len(list_of_QM_with_parameters)))  
        print('Parameters are given by s, d, f, r, v, t, u:')
        print(parameters)
        # optionally print all matrices explicitly
        if print_explicit_matrices:
            for quat_struct in list_of_QM_with_parameters:
                print('Quaternionic Matrix:')
                print_qm(quat_struct)
                #quat_struct.print_quaternionic_matrix()

def print_quaternionic_matrices_with_level(QM_dict, level, print_explicit_matrices = True):
    # this funtion prints all matrices of a dictionary of quaternionic structures along with their levels
    
    list_of_QM_with_level = [QM for QM in QM_dict[level]]
    print('Number of all structures: ' + str(len(list_of_QM_with_level)))   
    print('Level is given by: ' + str(level))
        
    if print_explicit_matrices:
        for quat_struct in list_of_QM_with_level:
            print('Quaternionic Matrix:')
            #quat_struct.print_quaternionic_matrix()
            print_qm(quat_struct)

def print_qm(QS):
    # this prints quaternionic matrices in a row by row format
    # optionally also print the completion
    for row in QS:
        print(row)

def save_dict(dict_to_save, filename):
    # inputs: dict_to_save - dictionary that should be saved; filename: name of the file without suffix
    # saves a dict in an .npy document in the current folder
    with open(filename + '.npy', 'wb') as f:
        np.save(f, dict_to_save)

def read_dict(filename):
    # loads a dict that was saved in filename.npy 
    return np.load(filename + '.npy', allow_pickle = 'TRUE').item()

def write_classification_results(filename, data, s, exec_time = None):
    
    with open(filename, 'w') as file:
        file.write('Level: ' + str(s) + '\n')
        file.write('Number of results: ' + str(len(data)) + '\n\n')
        for matrix in data:
            for line in matrix:
                file.write(str(line) + '\n')
            file.write('\n')
        if exec_time:
            file.write('Execution time: ' + time.strftime("%H:%M:%S", time.gmtime(exec_time)))

def write_parameter_results(ext_parameter, data, CPU_time, exec_time):
    file_name = str(ext_parameter)[1:-1].replace(', ', '_') + '.txt'
    
    with open(file_name, 'w') as file:
        file.write('Parameter: ' + str(ext_parameter[2:]) + '\n')
        if data:
            file.write('Number of results: ' + str(len(data)) + '\n\n')
            for matrix in data:
                for line in matrix:
                    file.write(str(line) + '\n')
                file.write('\n')
        else:
            file.write('Number of results: 0\n\n')

    append_time_stamp(file_name, CPU_time, exec_time)

def write_partial_parameter_results(data, qs_file, counter):
    file_name = qs_file + '_part_' + str(counter) + '.txt'
    
    with open(file_name, 'w') as file:
        #file.write('Parameter: ' + str(ext_parameter[2:]) + '\n')
        if data:
            file.write('Number of results: ' + str(len(data)) + '\n\n')
            for matrix in data:
                for line in matrix:
                    file.write(str(line) + '\n')
                file.write('\n')
        else:
            file.write('Number of results: 0\n\n')

def write_level_results(n, s, number_of_level_results, CPU_time, exec_time):
    outputs_all_parameters = glob.glob(str(n) + '_' + str(s) + '_*.txt')
    outputs_all_parameters.sort()

    file_name = str(n) + '_' + str(s) + '.txt'

    with open(file_name, 'w') as wfd:
        wfd.write('Level: ' + str(s) + '\n' + 'Number of results: ' + str(number_of_level_results) + '\n\n')

    with open(file_name, 'ab') as wfd:
        for f in outputs_all_parameters:
            with open(f, 'rb') as fd:
                shutil.copyfileobj(fd, wfd)
            os.remove(f)

    append_time_stamp(file_name, CPU_time, exec_time, 'Total Level Times:')

def write_all_results(n, number_of_results, CPU_time, exec_time, parameter_file = None, starting_string = None):
    if not starting_string:
        starting_string = str(n)
    
    partial_outputs_help = glob.glob(starting_string + '_*.txt')
    partial_outputs_help.sort()

    partial_outputs = [x for x in partial_outputs_help if x[-11:] != 'results.txt']
    

    if parameter_file:
        file_name = parameter_file + '_results.txt'
    else:
        file_name = str(n) + '.txt'

    with open(file_name, 'w') as wfd:
        wfd.write('Number of all results: ' + str(number_of_results) + '\n________________\n\n')
        
    with open(file_name, 'ab') as wfd:
        for f in partial_outputs:
            with open(f, 'rb') as fd:
                shutil.copyfileobj(fd, wfd)
            os.remove(f)

    append_time_stamp(file_name, CPU_time, exec_time, 'Total Times:')

def append_time_stamp(file_name, CPU_time, exec_time, additional_information = None):
    # this functions writes CPU and execution time into result files    
    with open(file_name, 'a') as file:
        if additional_information:
            file.write(additional_information + '\n')

        if additional_information == 'Total Level Times:':
            line_end = '\n________________________________\n\n'
        elif additional_information == 'Total Times:':
            line_end = '\n'
        else:
            line_end = '\n________\n\n'
            
        file.write('CPU time: ' + time.strftime("%d-%H:%M:%S", time.gmtime(CPU_time)) + '\n')
        file.write('Execution time: ' + time.strftime("%d-%H:%M:%S", time.gmtime(exec_time)) + line_end)


#### FUNCTIONS USED FOR THE CLASSIFICATION IN THE MAIN ROUTINE

def classify_quaternionic_structures(n, s_range, save_results = True, print_parameter_results = False, parallelize = False):
    # given a natural number n and range of levels s_range, classify all quaternionic structures of order n and level s in s_range
    # as not all basis transformations are considered, the number of matrices computed is an upper bound for the number of 
    # quaternionic structures of order n
    # optional inputs:
    # save_results to create a file containing the computed dictionary of quaternionic structures
    # print_parameter_results for output of the desired results (matrices and number thereof) and times
    # parallelize if different parameters shall be computed in parallel (not available on a windows system in this setting)
    # output: number of results for given cases

    starttime = time.time()
    start_CPU_time = time.process_time()

    number_of_level_results = {s: 0 for s in s_range}

    # levels to consider:
    for s in s_range: 
        # parameters to consider, this can be adapted if only a certain selection of parameters is supposed to be considered
        parameters = create_range_of_parameters(n, s)     

        # parallelization might not work on windows systems this way (cf. imports), there should be no problems with apple and linux
        if parallelize:        
            with Pool() as pool:
                results = pool.starmap(classify_QS, [(n, s, p) for p in parameters])
                    
            if results:
                number_of_level_results[s] += sum(len(x) if x is not None and x != [] else 0 for x in results)

            end_time = time.time()
            end_CPU_time = time.process_time()
                
            elapsed_time = end_time - starttime
            elapsed_CPU_time = end_CPU_time - start_CPU_time
        else: # single core computations
            for p in parameters:
                results = classify_QS(n, s, p)
                if results:
                    number_of_level_results[s] += len(results)

                end_time = time.time()
                end_CPU_time = time.process_time()
                
                elapsed_time = end_time - starttime
                elapsed_CPU_time = end_CPU_time - start_CPU_time
    
                if print_parameter_results:
                    if results:
                        for QS in results:
                            print_qm(QS)
        
                    print('CPU time:', time.strftime("%d-%H:%M:%S", time.gmtime(elapsed_CPU_time)))
                    print('Execution time:', time.strftime("%d-%H:%M:%S", time.gmtime(elapsed_time)))
       
        write_level_results(n, s, number_of_level_results[s], elapsed_CPU_time, elapsed_time)

    end_CPU_time = time.process_time()
    elapsed_time = time.time() - starttime
    elapsed_CPU_time = end_CPU_time - start_CPU_time

    print('CPU time:', time.strftime("%d-%H:%M:%S", time.gmtime(elapsed_CPU_time)))
    print('Execution time:', time.strftime("%d-%H:%M:%S", time.gmtime(elapsed_time)))

    number_of_results = 0
    for s in s_range:
        number_of_results += number_of_level_results[s]

    return number_of_results

def classify_QS(n = None, s = None, p = None, starting_QS = None, take_time = False):
    # input: order n, level s and list of parameters p, optionally take_time as stopwatch
    # this function computes a list of structures of given set of parameters

    starttime = time.time()
    start_CPU_time = time.process_time()
        
    qs_list_with_completion = []
    if debug_mode:
        partial_qs_list = []
        
    if not starting_QS:
        starting_QS = quaternionic_structure(n, s, p[0], p[1], p[2], p[3], p[4], p[5])
    else:
        n = starting_QS.n
    
    if starting_QS and starting_QS.validity:
        # if the starting_QS (either 2 by n or specific input) is valid, initiate an n by n matrix
        # the entry of this matrix at any relevant position (i, j) will be a tuple of 
        # the current quaternionic structure with matrix computed up to (i, j) and a list of possible values to extend row i
        # the classification matrix holds entry [None, []] for (i, j) < (1, n - 1) and for (i, j) s.t. i > j

        # starting indices, will be (1, n - 1) if we start with a 2 by n matrix as usual (no specific input)
        starting_row_ind = len(starting_QS.mat) - 1
        starting_col_ind = len(starting_QS.mat[-1]) - 1
        classification_matrix = [[[None, []] if [i, j] != [starting_row_ind, starting_col_ind] 
                                  else [starting_QS, []] if j == n - 1
                                  else [starting_QS, starting_QS.calc_possible_next_elements()] for j in range(n)] for i in range(n)]

        # current row/column indices:
        row_ind = starting_row_ind
        col_ind = starting_col_ind

        # the while loop computes quaternionic structures via depth search
        # whenever all options are exhausted at (row_ind, col_ind), go back to the previous entry where there are still entries to compute
        # this terminates when all possible values are considered, by going back to indices lexicographically smaller than the starting indices

        while row_ind > starting_row_ind or col_ind >= starting_col_ind:
            # debug_mode allows to collect partial results up to a certain depth in the matrix
            # the output files can later be used for more specific parallelization within a fixed parameter
            if debug_mode and row_ind == debug_index[0] and col_ind == debug_index[1]:
                classification_matrix[row_ind][col_ind][0].print_quaternionic_matrix()
                                
                endtime = time.time()
                end_CPU_time = time.process_time()            
                print('CPU time: ' + time.strftime("%d-%H:%M:%S", time.gmtime(end_CPU_time - start_CPU_time)))
                print('Execution time: ' + time.strftime("%d-%H:%M:%S", time.gmtime(endtime - starttime)))
                print('')
                
                partial_qs_list.append(copy.deepcopy(classification_matrix[row_ind][col_ind][0]))
                        
                del classification_matrix[row_ind][col_ind][0].mat[-1][-1]
                classification_matrix[row_ind][col_ind][0].downgrade_completion(False)
                col_ind -= 1                      
                continue
                
            # first case: we have a complete row
            if col_ind == n - 1:
                # in case of n by n matrix, we have a result and jump back to (n - 2, n - 2)
                if row_ind == n - 1:
                    # this deepcopy is necessary to get all results
                    qs_list_with_completion.append([copy.deepcopy(classification_matrix[row_ind][col_ind][0].mat), copy.deepcopy(classification_matrix[row_ind][col_ind][0].completion)])
                    row_ind -= 1
                    col_ind = n - 2
                    # downgrade quaternionic matrix and completion (necessary to avoid deepcopies)
                    del classification_matrix[row_ind][col_ind][0].mat[-1]
                    del classification_matrix[row_ind][col_ind][0].mat[-1][-1]
                    classification_matrix[row_ind][col_ind][0].downgrade_completion(True)
                else: # full row but no full matrix, so we can add another row
                    QS = classification_matrix[row_ind][col_ind][0].add_new_row()
                    
                    # if this QS is valid, put it into the classification matrix along with a list of possible extensions to the next entry of the new row
                    if QS and QS.validity:
                        row_ind += 1
                        col_ind = row_ind
                        classification_matrix[row_ind][col_ind] = [QS, QS.calc_possible_next_elements()]
                    else: # the new row is not valid, so we go back to the previous entry of the current row and thus do not change row_ind
                        col_ind -= 1
                        # calling 'add_new_row' changes the entry of the classification matrix, so we have to downgrade the row again
                        del classification_matrix[row_ind][col_ind][0].mat[-1]
                        del classification_matrix[row_ind][col_ind][0].mat[-1][-1]
                        classification_matrix[row_ind][col_ind][0].downgrade_completion(True)
            # now the case of incomplete rows, so we can add an element to the right
            else:
                # check whether there are possible values for extensions
                if classification_matrix[row_ind][col_ind][1]:
                    next_extension = classification_matrix[row_ind][col_ind][1][0]
                    QS = quaternionic_structure(QS = classification_matrix[row_ind][col_ind][0], 
                                                extension = next_extension)
                    # update list of possible values to check
                    classification_matrix[row_ind][col_ind][1].remove(next_extension)
                    # if QS is valid, go to the next column, else downgrade again                   
                    if QS and QS.validity:
                        col_ind += 1
                        classification_matrix[row_ind][col_ind] = [QS, QS.calc_possible_next_elements()]
                    else:
                        # delete last entry of the QS
                        del QS.mat[-1][-1]
                        QS.downgrade_completion(False)
                else: # there are no possible values left to check here, so we go back to the previous row or the previous column
                    if row_ind == starting_row_ind and col_ind == starting_col_ind:
                        # we have reached the exit point of the while loop
                        break
                    else:
                        if row_ind == col_ind:
                            row_ind -= 1
                            col_ind = n - 2
                            if row_ind == starting_row_ind and starting_col_ind == n - 1:
                                continue
                            # as before: downgrade matrix and completion
                            del classification_matrix[row_ind][col_ind][0].mat[-1]
                            del classification_matrix[row_ind][col_ind][0].mat[-1][-1]
                            classification_matrix[row_ind][col_ind][0].downgrade_completion(True)
                        else:
                            col_ind -= 1
                            # as before: downgrade matrix and completion
                            del classification_matrix[row_ind][col_ind][0].mat[-1][-1]
                            classification_matrix[row_ind][col_ind][0].downgrade_completion(False)
        
        endtime = time.time()
        end_CPU_time = time.process_time()
        
        elapsed_time = endtime - starttime
        elapsed_CPU_time = end_CPU_time - start_CPU_time

        if debug_mode:
            with open('debug_' + str(debug_index[0]) + '_' + str(debug_index[1]) + '_' + str([n] + [s] + p)[1:-1].replace(', ', '_') + '.obj', 'wb') as file:
                pickle.dump(partial_qs_list, file)

        # check whether there are any results, if yes: filter for reduced results only
        if qs_list_with_completion:
            qs_list = filter_for_reduced_matrices(qs_list_with_completion)
        else:
            qs_list = []

        if p:
            write_parameter_results([n] + [s] + p, qs_list, elapsed_CPU_time, elapsed_time)
            
        return qs_list

def filter_for_reduced_matrices(qs_list_with_completion):
    # input: non empty list of tuples of quaternionic matrices and their completions with identical parameters
    # this function detects all non reduced copies of elementary type quaternionic matrices
    # if the etc is not correct, this function might not filter copies of non elementary types
    # everything that is filtered out has the same zeros in the completion as another matrix, so we do not lose any results
    n = len(qs_list_with_completion[0][0])

    # initiate list of good matrices (only powers of 2 or elements in the span of the first row which means they are defined by linkage as a product of 2 powers) 
    # and list of bad matrices where another non 2 power appears which can correspond to this not being a product of 2 powers with linkage relations
    list_of_reduced_matrices_with_completion = []
    list_of_matrices_to_check = []

    # set maximal bit length that can appear in non 2 powers
    bit_border = biggest_exp_of_two(max(qs_list_with_completion[0][0][0][-1], 1))
    
    for x in qs_list_with_completion:
        # first two rows only contain zeros and powers of 2: row 0, 1 do not have to be considered
        # symmetry and diagonal rule: last row does not have to be considered
        for row_number in range(2, n - 1):
            # symmetry and diagonal rule: only check upper triangle
            for col_number in range(row_number + 1, n):
                current_bits = get_bits(x[0][row_number][col_number])
                # if entry at (row_number, col_number) is not a power of 2 and no product of powers of 2 in the first row, we have a bad matrix
                if len(current_bits) > 1 and current_bits[-1] > bit_border:
                    list_of_matrices_to_check.append(x)
                    break
            else:
                # this 'else-break-construction' assures that once we break the inner loop, we also break the middle loop
                continue
            break
        else:
            # we only get here if no violation has been found, so the matrix is reduced
            list_of_reduced_matrices_with_completion.append(x)   
        
    for matrix_to_check in list_of_matrices_to_check:
        # this boolean variable indicates whether we still have to check matrix_to_check or whether we have
        # found its reduced version in list_of_reduced_matrices_with_completion
        continue_checks = True
        for matrix_to_compare in list_of_reduced_matrices_with_completion:
            # only compare with the next matrix if we have not yet found matrix_to_check
            if continue_checks:
                # only applied for identical parameters so the first three rows of the completion are always the same
                for row_number in range(3, 2 ** n - 1):
                    # if the indices where zero occurs are the same in all rows, the matrices have the same reduced version
                    if set(matrix_to_check[1][row_number][0]) == set(matrix_to_compare[1][row_number][0]):
                        if row_number == 2 ** n - 2:
                            # this only happens if all rows of the completions are the same, so we have found a reduced version of matrix_to_check
                            continue_checks = False
                    else: 
                        break
        # if we still have continue_checks == True, then matrix_to_compare is not in the list of reduced matrices
        # this only happens if there is an unexpected result
        if continue_checks:
            print('There is an unexpected result:')
            matrix_to_check[0].print_quaternionic_matrix()
            list_of_reduced_matrices_with_completion.append(matrix_to_check)  

    return [x[0] for x in list_of_reduced_matrices_with_completion]


In [ ]:
# possible inputs: 
# n and s to classify order n for level s, i.e. '6 2' for order 6, level 2, or '6 1 2 4' for order 6 with all levels
# a .txt file containing parameters as lines, written as 'n s d f r v t u', the value for n always being the same
# an .obj file containing partial structures with identical parameters (for example, such a file can be obtained via debug mode)

# setting debug_mode = True enables to calculate all partial results up to debug_index which are then saved in an .obj file for further use
global debug_mode
debug_mode = False
global debug_index
debug_index = [0, 0]
# setting this option to True enables to parallelize for the different structures whenever the input is an .obj file
parallelize_for_single_input = False

if __name__ == "__main__":
    # initiate computation time and cpu time
    starttime = time.time()
    start_CPU_time = time.process_time()
 
    inp = str(input())
    if inp[-3:] == 'txt':
        # parameters to be computed can be given via a file
        parameter_file = inp[:-4]
        
        with open(parameter_file + '.txt') as file:
            parameter_lst = [[int(v) for v in line.rstrip().split()] for line in file]

        # The following means that for all parameters, we should have the same n 
        n = parameter_lst[0][0]

        number_of_results = 0

        # compute results for all given parameters
        for p in parameter_lst:
            results = classify_QS(p[0], p[1], p[2:])
            if results:
                number_of_results += len(results)
            
        endtime = time.time()
        end_CPU_time = time.process_time()

        elapsed_time = endtime - starttime
        elapsed_CPU_time = end_CPU_time - start_CPU_time

        write_all_results(n, number_of_results, elapsed_CPU_time, elapsed_time, parameter_file)
    elif inp[-3:] == 'obj':
        # parameters to be computed can be given via a file
        qs_file = inp[:-4]
        
        input_partial_qs = open(qs_file + '.obj', 'rb')
        qs_lst = pickle.load(input_partial_qs)
        input_partial_qs.close()

        # for an overview, print all matrices of the data from the .obj file
        print(len(qs_lst))
        
        for i in range(len(qs_lst)):
            print(i)
            qs_lst[i].print_quaternionic_matrix()
            print('')

        # note that all data should have the same n
        n = qs_lst[0].n

        number_of_results = 0
        counter_partial_results = 0

        if parallelize_for_single_input:        
            with Pool() as pool:
                results = pool.starmap(classify_QS, [(qs_lst[i].n, qs_lst[i].s, 
                                                      [qs_lst[i].d, qs_lst[i].f, qs_lst[i].r, qs_lst[i].v, qs_lst[i].t, qs_lst[i].u, i],
                                                      qs_lst[i]) for i in range(len(qs_lst))])
                    
            if results:
                number_of_results = sum(len(x) if x is not None and x != [] else 0 for x in results)
                
        else:
            for i in range(len(qs_lst)):
                qs = qs_lst[i]
                results = classify_QS(n = qs.n, s = qs.s, p = [qs.d, qs.f, qs.r, qs.v, qs.t, qs.u, i], starting_QS = qs)
                if results:
                    number_of_results += len(results)
                    write_partial_parameter_results(results, qs_file, counter_partial_results)
                    counter_partial_results += 1
            
        endtime = time.time()
        end_CPU_time = time.process_time()
        elapsed_time = endtime - starttime
        elapsed_CPU_time = end_CPU_time - start_CPU_time

        write_all_results(n, number_of_results, elapsed_CPU_time, elapsed_time, qs_file, starting_string = qs_file)
    else:
        # if no file is given, the input is interpreted as 'n s_range', with 'n' being interpreted as complete s_range 'n 1 2 4'
        data = [int(v) for v in inp.rstrip().split()]
        n = data[0]
        if len(data) > 1:
            s_range = data[1:]
        else:
            s_range = [1, 2, 4]
            
        # the boolean inputs can be changed to 'True' to (in the given order)
        # save the results (dictionary of QS) as an .npy file
        # print results for each parameter considered once it is computed
        # parallelize for all parameters
        number_of_results = classify_quaternionic_structures(n, s_range, False, False, False)

        # consider the cpu time and the execution time
        endtime = time.time()
        end_CPU_time = time.process_time()
        elapsed_time = endtime - starttime
        elapsed_CPU_time = end_CPU_time - start_CPU_time

        # write results and times into a .txt file
        write_all_results(n, number_of_results, elapsed_CPU_time, elapsed_time)      